In [ ]:
# Stocks Symbol List Data Source: https://datahub.io/core/nyse-other-listings#python
# Stock Data Source: FinnHub

In [1]:
# packages
# data
import finnhub
from datapackage import Package
import yfinance as yf

# scientific
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# utils
import time
import requests
import json



In [ ]:
################################################################################################

In [ ]:
# Gathering data from https://datahub.io/core/nyse-other-listings#python 
# -- on 01.12.2021 8497 symbols were captured
# stocks = []
# package = Package('https://datahub.io/core/nyse-other-listings/datapackage.json')

# # print list of all resources:
# # print(package.resource_names)

# # print processed tabular data (if exists any)
# for resource in package.resources:
#     if resource.descriptor['datahub']['type'] == 'derived/csv':
#         for entry in resource.read():
#             stocks.append(entry[0])

In [10]:
# EXTRA SYMBOLS
weed_stocks = ["GRWG", "JUSHF", "CRLBF", "HRVSF", "MJ", "CURLF", "CGC", "CRON", "MEDIF"]

tech_stocks = ["SNE", "XRX", "CSCO", "SIRI", "ROKU", "ATVI", "EA", "VZ", "T", "TMUS", "AMAT",
              "TXN", "MU", "FB", "MSFT", "SNAP", "AMZN", "TWTR", "AMD", "QCOM", "AVGO", "AAPL",
              "INTC", "NVDA", "NFLX", "ORCL", "IBM"]

fin_stocks = ["AXP", "V", "MA", "JPM", "DFS", "MS", "BAC", "GS", "SQ", "PYPL", "GDOT", "AMP"]

energy_stocks = ["NGG", "PCG", "RDS-B", "OXY", "MRO", "DD", "CVX", "DOW", "XOM", "PSX", "COP", 
              "HAL", "PLUG"]

travel_stocks = ["LUV", "MAR", "ABNB", "CCL", "RCL", "DIS", "AAL", "JBLU", "SAVE", "DAL", "HLT"]

food_consumer_stocks = ["DRI", "GIS", "NKE", "PG", "SHAK", "GRUB", "KDP", "PEP", "KO", "K", "QSR", "CMG",
              "YUM", "BGS", "ACI", "DAR", "KR", "GE", "GOOS", "HD", "LOW", "LULU", "DPZ", "SBUX", "MMM",
              "WMT", "TGT", "COST", "JACK", "MCD", "UBER", "GME"]

pharma_stocks = ["CVS", "RAD", "UNH", "LLY", "JNJ", "BMY", "AZN", "MRK", "ABBV", "GILD", "PFE"]

defense_infra_stocks = ["GD", "LMT", "NOC", "BA", "CAT", "WM"]

auto_stocks = ["HMC", "TM", "F", "TSLA", "GM"]

misc_stocks = ["ISR"]

all_stocks = (weed_stocks + tech_stocks + fin_stocks + energy_stocks 
                + travel_stocks + food_consumer_stocks + pharma_stocks 
                + defense_infra_stocks + auto_stocks + misc_stocks)

# pd.DataFrame(all_stocks).to_csv('./data/all_stocks.csv', index=False)

# excluding pharma
# all_stocks = (weed_stocks + tech_stocks + fin_stocks + energy_stocks 
#               + travel_stocks + food_consumer_stocks + defense_infra_stocks 
#               + auto_stocks + misc_stocks)

# for symb in all_stocks:
#     if str(symb) not in stocks:
#         stocks.append(str(symb))

In [ ]:
# scrubbing data of symbols which contain bad_chars = ["$", ";", ":", "!", "*", "."]
# -- run until there is no more output
# bad_chars = ["$", ";", ":", "!", "*", "."]

# for s in stocks:
#     for char in bad_chars:
#         if char in str(s):
#             print(s)
#             stocks.remove(str(s))

In [3]:
display(len(all_stocks))

127

In [ ]:
################################################################################################

In [18]:
df = {}
count = 1
repeat = []
s_p = pd.read_csv('/Users/erichenschel/Documents/Projects/blueAngelsWing/data/s_p_500.csv')
s_p_symb = s_p['Symbol'].to_list()
total = s_p_symb + all_stocks
display(len(total))

634

In [20]:
# Adding company beta and 10_day_avg_volume values to a dataframe

# Stock data
for stock in total:
# for stock in ["ABNB", ]:
    try:
        # request company's beta
        r = requests.get('https://finnhub.io/api/v1/stock/metric?symbol=' + str(stock) + '&metric=all&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()["metric"]
        count += 1

        beta = metrics["beta"]
        vol = metrics["10DayAverageTradingVolume"]
        if beta == None or vol == None or (str(stock) in df.keys()):
            continue

        # fill dictionary
        if beta >= 1.0:
            df[str(stock)] = {"beta": beta, "volume_10_day": vol}

    except:
        display(r.json())
        display(stock)
        repeat.append(stock)
        time.sleep(30)
        pass
print("repeating " + str(repeat))
for stock in repeat:
    try:
        # request company's beta
        r = requests.get('https://finnhub.io/api/v1/stock/metric?symbol=' + str(stock) + '&metric=all&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()["metric"]
        count += 1

        beta = metrics["beta"]
        vol = metrics["10DayAverageTradingVolume"]
        if beta == None or vol == None or (str(stock) in df.keys()):
            continue

        # fill dictionary
        if beta >= 1.0:
            df[str(stock)] = {"beta": beta, "volume_10_day": vol}

    except:
        display(r.json())
        display(stock)
        time.sleep(30)
        pass

{'error': 'API limit reached. Please try again later. Remaining Limit: 0'}

'BKR'

{'error': 'API limit reached. Please try again later. Remaining Limit: 0'}

'BLL'

{'error': 'API limit reached. Please try again later. Remaining Limit: 0'}

...

{'error': 'API limit reached. Please try again later. Remaining Limit: 0'}

'GILD'

{'metric': {},
 'metricType': 'all',
 'series': {},
 'symbol': '    Symbol                    Name                  Sector\n0      MMM              3M Company             Industrials\n1      AOS         A.O. Smith Corp             Industrials\n2      ABT     Abbott Laboratories             Health Care\n3     ABBV             AbbVie Inc.             Health Care\n4     ABMD             ABIOMED Inc             Health Care\n..     ...                     ...                     ...\n500    YUM         Yum! Brands Inc  Consumer Discretionary\n501   ZBRA      Zebra Technologies  Information Technology\n502    ZBH  Zimmer Biomet Holdings             Health Care\n503   ZION           Zions Bancorp              Financials\n504    ZTS                  Zoetis             Health Care\n\n[505 rows x 3 columns]'}

,Symbol,Name,Sector
0,MMM,3M Company,Industrials
1,AOS,A.O. Smith Corp,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie Inc.,Health Care
4,ABMD,ABIOMED Inc,Health Care
...,...,...,...
500,YUM,Yum! Brands Inc,Consumer Discretionary
501,ZBRA,Zebra Technologies,Information Technology
502,ZBH,Zimmer Biomet Holdings,Health Care
503,ZION,Zions Bancorp,Financials


KeyboardInterrupt: 

In [6]:
# dataframe to store stock data
data = pd.DataFrame(df)
data = data.transpose()
display(data)

,beta,volume_10_day
GRWG,3.11377,2.53634
CURLF,1.57790,0.97750
CGC,2.37242,1.44393
CRON,2.11014,1.05673
XRX,1.87977,2.58729
...,...,...
F,1.27192,125.95161
TSLA,2.13589,34.20522
GM,1.38764,38.65255
ISR,2.27569,19.66579


In [7]:
data['c'] = ''
data['o'] = ''
data['adx'] = ''
data['signal'] = ''
data['trending'] = ''
data['shareOutstanding'] = ''

In [8]:
# adding trends, signals, and price levels
count = 1
repeat = []

# Stock data
# for stock in ["MA", "CURLF", "AXP", "COP", "HAL", "DAR", "GOOS"]:
for stock in data.index:
    try:
        # Aggregate Indicators -- (signal, adx, trending)
        r = requests.get('https://finnhub.io/api/v1/scan/technical-indicator?symbol=' + str(stock) + '&resolution=W&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()
        count += 1

        # trending metrics
        signal = metrics["technicalAnalysis"]["signal"]
        adx = metrics["trend"]["adx"]
        trending = metrics["trend"]["trending"]
        # add to dataframe
        data.at[str(stock), 'adx'] = adx
        data.at[str(stock), 'signal'] = signal
        data.at[str(stock), 'trending'] = trending


        # company profile -- shareOutstanding 
        r = requests.get('https://finnhub.io/api/v1/stock/profile2?symbol=' + str(stock) + '&token=bvt5fmf48v6rku8bvmn0')
        shares = r.json()["shareOutstanding"]
        # add to dataframe
        data.at[str(stock), 'shareOutstanding'] = shares

        # open price and current price
        r = requests.get('https://finnhub.io/api/v1/quote?symbol=' + str(stock) + '&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()
        # add to dataframe
        data.at[str(stock), 'o'] = metrics['o']
        data.at[str(stock), 'c'] = metrics['c']

        # News Sentiment
        r = requests.get('https://finnhub.io/api/v1/news-sentiment?symbol=' + str(stock) + '&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()
        # add to dataframe
        data.at[str(stock), 'bearish'] = metrics["sentiment"]["bearishPercent"]
        data.at[str(stock), 'bullish'] = metrics["sentiment"]["bullishPercent"]

    except:
        print(str(count) + '...' + str(stock))
        time.sleep(30)
        repeat.append(stock)
        pass
    
print("repeating: " + str(repeat))
for stock in repeat:
    try:
        # Aggregate Indicators -- (signal, adx, trending)
        r = requests.get('https://finnhub.io/api/v1/scan/technical-indicator?symbol=' + str(stock) + '&resolution=W&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()
        count += 1

        signal = metrics["technicalAnalysis"]["signal"]
        adx = metrics["trend"]["adx"]
        trending = metrics["trend"]["trending"]

        # add to dataframe
        data.at[str(stock), 'adx'] = adx
        data.at[str(stock), 'signal'] = signal
        data.at[str(stock), 'trending'] = trending


        # company profile -- shareOutstanding 
        r = requests.get('https://finnhub.io/api/v1/stock/profile2?symbol=' + str(stock) + '&token=bvt5fmf48v6rku8bvmn0')
        shares = r.json()["shareOutstanding"]

        # add to dataframe
        data.at[str(stock), 'shareOutstanding'] = shares


        # open price and current price
        r = requests.get('https://finnhub.io/api/v1/quote?symbol=' + str(stock) + '&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()

        # add to dataframe
        data.at[str(stock), 'o'] = metrics['o']
        data.at[str(stock), 'c'] = metrics['c']

        # News Sentiment
        r = requests.get('https://finnhub.io/api/v1/news-sentiment?symbol=' + str(stock) + '&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()

        data.at[str(stock), 'bearish'] = metrics["sentiment"]["bearishPercent"]
        data.at[str(stock), 'bullish'] = metrics["sentiment"]["bullishPercent"]

    except:
        print(str(count) + '...' + str(stock))
        time.sleep(30)
        pass

5...CRON
17...AVGO
33...OXY
34...MRO
50...HLT
51...DRI
67...F
68...TSLA
70...ISR
repeating: ['CRON', 'AVGO', 'OXY', 'MRO', 'HLT', 'DRI', 'F', 'TSLA', 'ISR']
72...CRON
80...ISR


In [38]:
# data = data.sort_values(by=["signal", "beta", "shareOutstanding"], axis=0, ascending=False)
data = data.sort_values(by=["adx"], axis=0, ascending=False)
data.to_csv('./data/02_01_21_0107.csv', index=True)
# data = data.drop("CRON", axis=0)
n = len(data)
step = 15

display(data.head(30).sort_values(by=["beta"], axis=0, ascending=False))

,beta,volume_10_day,c,o,adx,signal,trending,shareOutstanding,bearish,bullish
GRWG,3.11377,2.53634,43.2299,42.365,70.9655,buy,True,36.88,0.0000,1.0000
SQ,2.56729,8.51036,215.29,214.22,39.6073,neutral,True,432.8,0.0000,1.0000
CGC,2.37242,1.44393,39.95,40.41,38.2949,strong buy,True,350.11,0.0000,1.0000
ISR,2.27569,19.66579,1.7199,1.7199,39.2617,buy,True,68.9,NaN,NaN
AMD,2.18109,55.18614,85.58,85.46,43.0968,buy,True,1202.71,0.0000,1.0000
TSLA,2.13589,34.20522,792.64,784.05,56.6435,buy,True,947.9,0.5676,0.4324
ROKU,1.94542,4.12447,388.025,384.63,53.8478,buy,True,119.9,0.0000,1.0000
PLUG,1.87740,73.78250,63.365,63.5101,61.9773,buy,True,303.38,0.5000,0.5000
SHAK,1.78637,1.17170,112.62,112.74,50.6535,buy,True,37.56,0.5000,0.5000
DFS,1.75996,3.40184,83.625,83.09,36.8049,buy,True,307,0.5000,0.5000


In [ ]:
# for DEV
# pattern recognition
# for stock in data_1.index:
#     r = requests.get('https://finnhub.io/api/v1/scan/pattern?symbol=' + str(stock) + '&resolution=D&token=bvt5fmf48v6rku8bvmn0')
#     points = r.json()["points"]
#     for entry in points:
#         display(entry)